In [1]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
quandl.ApiConfig.api_key = 'iB1vwTHbXzBdG755yy2W'

In [8]:
class Macro_Series:
    def __init__(self,name,dataframe):
        print(name)
        self.name = name
        self.df = dataframe
        
        time_data = self.get_df_date_info(dataframe)
        self.direction = time_data[0]
        self.start_date = time_data[1]
        self.end_date = time_data[2]
    
    ## returns direction of a time sorted df from earliest time -> more recent time, start date and end date.
    def get_df_date_info(self,series):
        d1 = series.iloc[0].name
        d2 = series.iloc[-1].name
        if d1 < d2:
            #low -> high
            return [1, d1, d2]
        if d2 < d1:
            #high -> low
            return [-1, d2, d1]

In [9]:
def get_macro_data():
    macro_data = {}
    macro_data['us_gdp_potential'] = Macro_Series( 'us_gdp_potential',quandl.get('FRED/GDPPOT', column_index='1') )
    macro_data['us_nat_unemployment'] = Macro_Series( 'us_nat_unemployment',quandl.get('FRED/NROUST',column_index='1') )

    pred_gov_net_lending_keys = {'FRED/GGNLBPDEA188N':'Germany','FRED/GGNLBPFRA188N':'France','FRED/GGNLBPJPA188N':'Japan','FRED/GGNLBPCAA188N':'Canada',
                            'FRED/GGNLBPCNA188N':'China','FRED/GGNLBPRUA188N':'Russia','FRED/GGNLBAGBA188N':'UK','FRED/GGNLBPEZA188N':'EU',
                            'FRED/GGNLBPUSA188N':'US','FRED/GGNLBAINA188N':'India'}

    pred_gov_net_lending = []
    for i in pred_gov_net_lending_keys:
        name = pred_gov_net_lending_keys[i] + '_pred_gov_net_lending'
        pred_gov_net_lending.append( Macro_Series(name,quandl.get(i,column_index='1') ))
    macro_data['pred_gov_net_lending'] = pred_gov_net_lending
    
    macro_data['fed_total_public_debt'] = Macro_Series( 'fed_total_public_debt', quandl.get('FRED/GFDEBTN',column_index='1') )

    gov_gross_debt_keys = {'FRED/GGGDTADEA188N':'Germany','FRED/GGGDTAFRA188N':'France','FRED/GGGDTAJPA188N':'Japan','FRED/GGGDTACAA188N':'Canada',
                           'FRED/GGGDTACNA188N':'China','FRED/GGGDTARUA188N':'Russia','FRED/GGGDTAGBA188N':'UK','FRED/GGGDTAEZA188N':'EU',
                           'FRED/GGGDTAUSA188N':'US','FRED/GGGDTAINA188N':'India',}
    gov_gross_debt = []
    for i in gov_gross_debt_keys:
        name = gov_gross_debt_keys[i] + '_gov_gross_debt'
        gov_gross_debt.append( Macro_Series(name,quandl.get(i,column_index='1')) )
    macro_data['gov_gross_debt'] = gov_gross_debt
                                                       
    macro_data['gold_price'] = Macro_Series('gold_price', quandl.get('LBMA/GOLD',column_index='1'))
    return macro_data
    

In [10]:
mc_data = get_macro_data()

us_gdp_potential
us_nat_unemployment
Germany_pred_gov_net_lending
France_pred_gov_net_lending
Japan_pred_gov_net_lending
Canada_pred_gov_net_lending
China_pred_gov_net_lending
Russia_pred_gov_net_lending
UK_pred_gov_net_lending
EU_pred_gov_net_lending
US_pred_gov_net_lending
India_pred_gov_net_lending
fed_total_public_debt
Germany_gov_gross_debt
France_gov_gross_debt
Japan_gov_gross_debt
Canada_gov_gross_debt
China_gov_gross_debt
Russia_gov_gross_debt
UK_gov_gross_debt
EU_gov_gross_debt
US_gov_gross_debt
India_gov_gross_debt
gold_price


In [6]:
def analyse_macro_data(macro_data):
    print("Length:",len(macro_data))
    for i in macro_data:
        print(i)
        print(type(macro_data[i]))
        if type(macro_data[i]) == type(pd.DataFrame()):
            print('DateFrame')
        else:
            print('List')
    return

In [7]:
analyse_macro_data(mc_data)

Length: 6
us_gdp_potential
<class '__main__.Macro_Series'>
List
us_nat_unemployment
<class '__main__.Macro_Series'>
List
pred_gov_net_lending
<class 'list'>
List
fed_total_public_debt
<class '__main__.Macro_Series'>
List
gov_gross_debt
<class 'list'>
List
gold_price
<class '__main__.Macro_Series'>
List
